![title](images/logo_datacraft.png)

<h1><center>Ethical AI</center></h1>
<center><i>Datacraft workshop, sponsored by La French Tech</i></center>


**Objective of the notebook**: understand how to tackle the fairness challenge in AI on real use case - credit granting

This exercise highly relies on the work conducted by the Dalex team (see [here](https://dalex.drwhy.ai/), [here](https://dalex.drwhy.ai/python-dalex-fairness.html) or [here](https://dalex.drwhy.ai/python-dalex-fairness2.html)). It is based on the German Credit dataset. This dataset is often used to benchmark fairness approaches.

Our humble ambition is to combine the power of different tools: Dalex as a base, complemented by other tools from the open source community, like [AIF360](https://aif360.mybluemix.net/).

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Prerequesites" data-toc-modified-id="Prerequesites-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Prerequesites</a></span></li><li><span><a href="#Bias-a-priori" data-toc-modified-id="Bias-a-priori-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Bias a priori</a></span></li><li><span><a href="#Dataset" data-toc-modified-id="Dataset-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Dataset</a></span></li><li><span><a href="#EDA" data-toc-modified-id="EDA-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>EDA</a></span></li><li><span><a href="#ML-prerequisites" data-toc-modified-id="ML-prerequisites-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>ML prerequisites</a></span></li><li><span><a href="#Fairness-evaluation-principles" data-toc-modified-id="Fairness-evaluation-principles-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Fairness evaluation principles</a></span></li><li><span><a href="#Strategies" data-toc-modified-id="Strategies-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Strategies</a></span><ul class="toc-item"><li><span><a href="#Do-nothing" data-toc-modified-id="Do-nothing-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Do nothing</a></span></li><li><span><a href="#-Fairness-evaluation-" data-toc-modified-id="-Fairness-evaluation--7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span> Fairness evaluation </a></span><ul class="toc-item"><li><span><a href="#-With-dalex-" data-toc-modified-id="-With-dalex--7.2.1"><span class="toc-item-num">7.2.1&nbsp;&nbsp;</span> With dalex </a></span></li><li><span><a href="#-With-aif360-" data-toc-modified-id="-With-aif360--7.2.2"><span class="toc-item-num">7.2.2&nbsp;&nbsp;</span> With aif360 </a></span></li></ul></li></ul></li><li><span><a href="#Preprocessing" data-toc-modified-id="Preprocessing-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Preprocessing</a></span></li><li><span><a href="#Inprocessing" data-toc-modified-id="Inprocessing-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Inprocessing</a></span></li><li><span><a href="#Postprocessing" data-toc-modified-id="Postprocessing-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Postprocessing</a></span></li></ul></div>

## Prerequesites

In [1]:
# We strongly recommend to install the necessary libraries from a dedicated virtual environment
# See Minicoda e.g.: https://docs.conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html#creating-an-environment-with-commands

! pip install fairlearn
! pip install dalex -U
! pip install -U scikit-learn
! pip install -U pandas
! pip install aif360
! pip install -U plotly

In [2]:
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.tree import DecisionTreeClassifier

import aif360
import dalex as dx
import numpy as np
import pandas as pd
import sklearn

from aif360.datasets import BinaryLabelDataset
from aif360.metrics import ClassificationMetric

{
    "aif360": aif360.__version__,
    "dalex": dx.__version__,
    "numpy": np.__version__,
    "pandas": pd.__version__,
    "sklearn": sklearn.__version__
}

{'aif360': '0.4.0',
 'dalex': '1.4.1',
 'numpy': '1.20.3',
 'pandas': '1.3.4',
 'sklearn': '1.0.1'}

In [3]:
sklearn.set_config(display="diagram")

## Bias a priori

*When implementing an AI system, fairness and biases must be an important component during conception, especially when dealing with sensitive information, and/or Personally Identifiable Information (PII), and/or Personal Health Information (PHI). Indeed, not only those information are bound to the law (GDPR in Europe e.g.), but they are also bound to a brand image challenge.*

Today's challenge aims at **assigning a risk to credit-seeker**.

Before implementing any AI system to predict the likelihood of a credit-seeker to be granted a credit, **AI engineers AND business stakeholders** should **sit and identify potential sources of biases**.

In this case, potential biases might lie in:

- Genre
- Age
- Revenue?

## Dataset

In [4]:
df, target = dx.datasets.load_german(), "risk"

## EDA

In [5]:
def facets(df: pd.DataFrame):
    """ Displays Facets viz
    Source: https://colab.research.google.com/github/PAIR-code/facets/blob/master/colab_facets.ipynb#scrollTo=XtOzRy8Z3M36
    
    Parameters
    ----------
    df: pd.DataFrame
        Data to plot
    """
    from IPython.core.display import display, HTML
    jsonstr = df.to_json(orient='records')
    HTML_TEMPLATE = """
            <script src="https://cdnjs.cloudflare.com/ajax/libs/webcomponentsjs/1.3.3/webcomponents-lite.js"></script>
            <link rel="import" href="https://raw.githubusercontent.com/PAIR-code/facets/1.0.0/facets-dist/facets-jupyter.html">
            <facets-dive id="elem" height="600"></facets-dive>
            <script>
              var data = {jsonstr};
              document.querySelector("#elem").data = data;
            </script>"""
    html = HTML_TEMPLATE.format(jsonstr=jsonstr)
    display(HTML(html))

In [ ]:
facets(df)

In [ ]:
(
    df
    .groupby("sex")
    ["risk"]
    .value_counts(normalize=True)
    .multiply(100)
    .astype(int)
    .to_frame()
)

In [ ]:
(
    df
    .assign(youth=lambda x: np.where(x["age"] <= 30, "young", "old"))
    .groupby("youth")
    ["risk"]
    .value_counts(normalize=True)
    .multiply(100)
    .astype(int)
    .to_frame()
)

In [ ]:
(
    df
    .assign(youth=lambda x: np.where(x["age"] <= 30, "young", "old"))
    .groupby(["sex", "youth"])
    ["risk"]
    .value_counts(normalize=True)
    .multiply(100)
    .astype(int)
    .to_frame()
)

**Quick EDA**

If we bin by sex, we realize that women are:
- Under represented in volume (310 vs 690)
- Slightly biased from a credit-granting perspective (64% vs 72%)

If we bin by age, we realize that young are:
- Under represented in volume (411 VS 589)
- Slightly biased from a credit-granting perspective (63% vs 74%)

If we combine the two, biases in input data are amplified.

**Conclusion: the apriori from section 2 looks confirmed and will need to be carefully handled during modelling.**

----

**Notes**

1. This is a toy example where biases are "straightforward" and well identified as "recurrent" social biases. However, it might not always be as easy to detect them. Additional sources might come from data history, selection bias, data incompleteness, unexpected sources of bias (column/attribute), ...
2. In this example, we identified biases related to representation in volume. If we had not been exposed to such discrepencies, namely having a balanced dataset, could we have concluded that biases would have been limited while modelling? Not so sure, see [this article](https://arxiv.org/pdf/1811.08489.pdf).

## ML prerequisites

In [36]:
df["job"] = df["job"].astype("object")
df["sex"] = np.where(df["sex"] == "female", 1, 0).astype(np.int64)

In [37]:
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(columns=[target]),
    df[target],
    test_size=0.3,
    random_state=42
)

## Fairness evaluation principles

![title](images/dalex_pipeline.jpg)

The main object of dalex is the `Explainer` container which wraps a **dataset** (features and target) and a **trained model**. 

Once the data and the model have been wrapped, one needs to fix **protected and privileged attributes**.

**Important note**: beware these choices correspond to an a priori understanding of the problem and could miss hidden flaws of the model. An interesting line of work would consist in conducting a kind of grid-search exploration for potential biases.

In [38]:
# protected = df.sex.astype("str") + '_' + np.where(df.age < 25, 'young', 'old')
protected = X_test.sex.astype("str") + '_' + np.where(X_test.age < 25, 'young', 'old')
privileged = '0_old'

## Strategies

### Do nothing

In [39]:
preprocessor = make_column_transformer(
      ("passthrough", make_column_selector(dtype_include=np.number)),
      (OneHotEncoder(handle_unknown="ignore"), make_column_selector(dtype_include=object))
)

clf_decisiontree = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', DecisionTreeClassifier(max_depth=7, random_state=123))
])

In [40]:
# clf_decisiontree.fit(df.drop(columns=[target]), df[target])
clf_decisiontree.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('passthrough', 'passthrough',
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001F428C5F508>),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001F428C5F908>)])),
                ('classifier',
                 DecisionTreeClassifier(max_depth=7, random_state=123))])

In [47]:
# exp_decisiontree = dx.Explainer(clf_decisiontree, df.drop(columns=[target]), df[target], verbose=False)
exp_decisiontree = dx.Explainer(clf_decisiontree, X_test, y_test, verbose=False)

In [48]:
exp_decisiontree.model_performance().result

,recall,precision,f1,accuracy,auc
DecisionTreeClassifier,0.832536,0.759825,0.794521,0.7,0.691861


The method `model_fairness` returns a fairness object from which fairness evaluations can be conducted. Notice that every metrics inherited from the confusion matrix are computed during the instantiation.

Two methods can then be performed:
- The `fairness_check` method, which returns a report on the fairness of the model. It requires an epsilon parameter that corresponds to the threshold ratio below which a given metric is considered to be unfair (default value is 0.8).
- The `plot` method, which allows to visualize the main fairness ratios between the protected subgroups and the privileged one.

In [49]:
fairness_decisiontree = exp_decisiontree.model_fairness(protected=protected, privileged=privileged)

In [50]:
fairness_decisiontree.fairness_check(epsilon = 0.8) # default epsilon

Bias detected in 3 metrics: TPR, FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on '0_old'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
             TPR       ACC       PPV       FPR       STP
0_young  0.55991  0.825485  1.002632  0.355114  0.481526


In [51]:
fairness_decisiontree.plot()

<h3> Fairness evaluation </h3>

<h4> With aif360 </h3>

With aif30, there is no such thing as a wrapper of a dataset together with a model. Instead, we need to define two instances of the class ```BinaryLabelDataset```:
<ul>
    <li> One with the groundtruth labels and the protected (resp. privileged) subgroups, </li>
    <li> One with the predicted labels (obtained from a trained model) and the protected (resp. privileged) subgroups.</li>
</ul>
Beware that the data need to be np.arrays.

In [ ]:
data_for_aif = X_test[['sex']]
data_for_aif['sex'] = data_for_aif['sex'].apply(lambda x: int(x == 'male'))
data_for_aif['age'] = X_test['age'].apply(lambda x: int(x >= 25))

data_for_aif_gt = data_for_aif.copy()
data_for_aif_gt['risk'] = y_test

data_for_aif_pred = data_for_aif.copy()
data_for_aif_pred['risk'] = clf_logreg.predict(X_test)

binary_dataset_gt = BinaryLabelDataset(df=data_for_aif_gt, label_names=['risk'], protected_attribute_names=['sex', 'age'])
binary_dataset_pred = BinaryLabelDataset(df=data_for_aif_pred, label_names=['risk'], protected_attribute_names=['sex', 'age'])

The two BinaryLabelDataset are then wrapped into a ```ClassificationMetric``` object, which requires to fix privileged and unprivileged subgroups in the form of dictionaries. From this object, several fairness metrics can be computed. </br>
Warning: It seems that, although several groups can be defined, the metrics are only computed for the two first one (an issue of the librabry ?).

In [ ]:
aif_classif_metrics = ClassificationMetric(binary_dataset_gt, 
                                           binary_dataset_pred, 
                                           privileged_groups=[{'sex': 1.0, 'age': 1.0}],
                                           unprivileged_groups=[{'sex': 1.0, 'age': 0.0}, {'sex': 0.0, 'age': 1.0}, {'sex': 0.0, 'age': 0.0}]
                                          )

In [ ]:
aif_classif_metrics.disparate_impact()

## Preprocessing

## Inprocessing

## Postprocessing